# 02 - Gentle Roaster 

This notebook implements our most constructive CV critique model.

## Characteristics
- Generally friendly
- Focuses on  improvements & highlights strengths before weaknesses
- Uses lower temperature 0.3-0.5

---

In [1]:
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import sys
sys.path.append('..')
import google.generativeai as genai

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

An error occurred: module 'importlib.metadata' has no attribute 'packages_distributions'


/Users/hannokuegler/Library/CloudStorage/OneDrive-WUWien/SBWL/Data Science/4_LLM/roast_my_cv/roast_my_cv/.venv1/lib/python3.9/site-packages/google/api_core/_python_version_support.py:252: FutureWarning: You are using a Python version (3.9.6) past its end of life. Google will update google.api_core with critical bug fixes on a best-effort basis, but not with any other fixes or features. Please upgrade to the latest Python version, or at least Python 3.10, and then update google.api_core.
  warnings.warn(message, FutureWarning)
/Users/hannokuegler/Library/CloudStorage/OneDrive-WUWien/SBWL/Data Science/4_LLM/roast_my_cv/roast_my_cv/.venv1/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Setup

- Loading API from config.py (only saved locally)

In [2]:
# Load API key from config.py
import sys
sys.path.append('..')
from config import GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)
print("API key loaded from config.py")

API key loaded from config.py


## Load Data and Helper Functions

In [3]:
# Load dataset
df = pd.read_csv('../data/resume_data.csv')

# Load test CV indices
with open('../data/test_cv_indices.json', 'r') as f:
    test_data = json.load(f)
    test_cv_indices = test_data['indices']

print(f"Loaded {len(df)} resumes")
print(f"Test CVs: {test_cv_indices}")

Loaded 9544 resumes
Test CVs: [0, 1]


In [4]:
# CV formatting function (from EDA notebook)
def format_cv_for_llm(resume_row):
    """
    Format a resume row into a readable text for LLM processing.
    """
    cv_text = []
    
    if pd.notna(resume_row.get('career_objective')):
        cv_text.append(f"CAREER OBJECTIVE:\n{resume_row['career_objective']}")
    
    if pd.notna(resume_row.get('skills')):
        cv_text.append(f"\nSKILLS:\n{resume_row['skills']}")
    
    education_parts = []
    if pd.notna(resume_row.get('educational_institution_name')):
        education_parts.append(f"Institution: {resume_row['educational_institution_name']}")
    if pd.notna(resume_row.get('degree_names')):
        education_parts.append(f"Degree: {resume_row['degree_names']}")
    if pd.notna(resume_row.get('major_field_of_studies')):
        education_parts.append(f"Major: {resume_row['major_field_of_studies']}")
    if pd.notna(resume_row.get('passing_years')):
        education_parts.append(f"Year: {resume_row['passing_years']}")
    
    if education_parts:
        cv_text.append(f"\nEDUCATION:\n" + "\n".join(education_parts))
    
    work_parts = []
    if pd.notna(resume_row.get('professional_company_names')):
        work_parts.append(f"Company: {resume_row['professional_company_names']}")
    if pd.notna(resume_row.get('positions')):
        work_parts.append(f"Position: {resume_row['positions']}")
    if pd.notna(resume_row.get('start_dates')):
        work_parts.append(f"Period: {resume_row['start_dates']}")
        if pd.notna(resume_row.get('end_dates')):
            work_parts.append(f" to {resume_row['end_dates']}")
    if pd.notna(resume_row.get('responsibilities')):
        work_parts.append(f"Responsibilities:\n{resume_row['responsibilities']}")
    
    if work_parts:
        cv_text.append(f"\nWORK EXPERIENCE:\n" + "\n".join(work_parts))
    
    if pd.notna(resume_row.get('languages')):
        cv_text.append(f"\nLANGUAGES:\n{resume_row['languages']}")
    
    if pd.notna(resume_row.get('certification_skills')):
        cv_text.append(f"\nCERTIFICATIONS:\n{resume_row['certification_skills']}")
    
    return "\n".join(cv_text)

## Gentle Roaster Prompt Design

In [5]:
GENTLE_SYSTEM_PROMPT = """You are a kind and supportive career advisor who helps people improve their resumes.

Your approach:
1. Start with genuine positive observations about their CV
2. Provide constructive, specific feedback
3. Suggest concrete improvements
4. End with encouragement

Keep your feedback:
- Professional and friendly
- Specific and actionable
- Balanced (acknowledge strengths)
- Encouraging and motivating

Structure your response:
 STRENGTHS: What's working well
 AREAS FOR IMPROVEMENT: Specific suggestions
 ACTION ITEMS: Clear next steps
 FINAL THOUGHTS: Encouraging message
"""

def create_gentle_prompt(cv_text):
    """Create a gentle roasting prompt."""
    return f"""Please review this CV and provide constructive, encouraging feedback:

{cv_text}

Provide your feedback following the structure in the system prompt."""

## Temperature Tuning Experiments

We test different temperature settings to find the optimal setting for for "gentle" feedback.

In [6]:
def roast_cv(cv_text, temperature=0.4, model_name="gemini-2.0-flash"):
    """
    Generate CV critique using Gemini.
    
    Args:
        cv_text: Formatted CV text
        temperature: Controls randomness (0.0-1.0)
        model_name: Gemini model to use
    
    Returns:
        str: Generated critique
    """
    model = genai.GenerativeModel(
        model_name=model_name,
        generation_config=genai.GenerationConfig(
            temperature=temperature,
            top_p=0.95,
            top_k=40,
            max_output_tokens=1024,
        )
    )
    
    # Combine system prompt and user prompt
    full_prompt = f"{GENTLE_SYSTEM_PROMPT}\n\n{create_gentle_prompt(cv_text)}"
    
    response = model.generate_content(full_prompt)
    return response.text

# Test with first CV
test_cv = format_cv_for_llm(df.iloc[test_cv_indices[0]])
print("Test CV:")
print("="*80)
print(test_cv[:500] + "...")
print("="*80)

Test CV:
CAREER OBJECTIVE:
Big data analytics working and database warehouse manager with robust experience in handling all kinds of data. I have also used multiple cloud infrastructure services and am well acquainted with them. Currently in search of role that offers more of development.

SKILLS:
['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapreduce', 'Spark', 'Java', 'Machine Learning', 'Cloud', 'Hdfs', 'YARN', 'Core Java', 'Data Science', 'C++', 'Data Structures', 'DBMS', 'RDBMS', 'Informatica', 'Talend...


### Experiment 1: Low Temperature (0.3)
More focused and consistent

In [7]:
print(" Temperature: 0.3 (Most Conservative)")
print("="*80)
result_temp_03 = roast_cv(test_cv, temperature=0.3)
print(result_temp_03)
print("\n" + "="*80)

 Temperature: 0.3 (Most Conservative)
Okay, I'd be happy to help you refine your CV! Here's my feedback, focusing on how to make it even stronger and more impactful:

**STRENGTHS:**

*   You have a good foundation of technical skills. Listing them out like this is a great way to get past the ATS (Applicant Tracking System).
*   You've clearly identified your desired career path (development-focused role).
*   Including your work experience at Coca-Cola is excellent, and the listed responsibilities provide a glimpse into your contributions.

**AREAS FOR IMPROVEMENT:**

*   **Career Objective:** The current objective is a bit generic. It could be more targeted and compelling. Instead of just stating what you *are*, try to convey what you *want to achieve* and *how you can benefit the company*.
*   **Skills:** While the list is comprehensive, it lacks context. Consider adding a brief proficiency level (e.g., "Proficient in Python," "Experienced with Hadoop") or, even better, integrating t

### Experiment 2: Medium-Low Temperature (0.5)
More creative

In [8]:
print(" Temperature: 0.5 (Balanced)")
print("="*80)
result_temp_05 = roast_cv(test_cv, temperature=0.5)
print(result_temp_05)
print("\n" + "="*80)

 Temperature: 0.5 (Balanced)
Okay, I'd be happy to help you refine your CV! Here's my feedback, focusing on how we can make it even stronger:

**STRENGTHS:**

*   You have a solid foundation of technical skills. Listing them out like this gives a clear snapshot of your expertise in areas like Big Data, Cloud technologies, and programming languages.
*   The inclusion of your education, even though recent, is good. It provides context for your skill set.
*   Listing responsibilities at Coca-Cola is a good start to help showcase your experience.

**AREAS FOR IMPROVEMENT:**

*   **Career Objective:** The current objective is a bit generic. It tells me *what* you are, but not *why* a company should hire you. It also contains grammatical errors.
*   **Skills:** While the list is comprehensive, it lacks context. Are you proficient in all of these? How have you used them?
*   **Work Experience:** The responsibilities are quite generic. They could apply to many different roles. We need to make 

### Experiment 3: Medium Temperature (0.7)
Very creative

In [9]:
print(" Temperature: 0.7 (More Creative)")
print("="*80)
result_temp_07 = roast_cv(test_cv, temperature=0.7)
print(result_temp_07)
print("\n" + "="*80)

 Temperature: 0.7 (More Creative)
Okay, I'd be happy to review your CV and provide some constructive feedback! Here's my assessment:

**STRENGTHS:**

*   You have a good range of technical skills listed, which is great for immediately showcasing your expertise to potential employers.
*   Listing your work experience at Coca-Cola, a well-known company, immediately catches the reader's attention.
*   The skills section provides a clear and concise overview of your technical capabilities.
*   Listing the technologies in your skills section is great for applicant tracking systems (ATS) and recruiters who are searching for candidates with specific skills.

**AREAS FOR IMPROVEMENT:**

*   **Career Objective:** The career objective is a bit generic and could be more impactful. It focuses on what you *have* done rather than what you *want* to do and *how* you can benefit the company.
*   **Skills:** While the list is comprehensive, it lacks context. Consider adding a brief description of your 

## Select Optimal Temperature

Based on reading through the experiments above, we manually select the temperature that provides the best balance of:
- Professional tone, Specific feedback, Encouraging message and Consistency

**Recommended: 0.4 for gentle roasting**

In [22]:
# Set optimal temperature
OPTIMAL_TEMPERATURE = 0.4

print(f" Selected optimal temperature: {OPTIMAL_TEMPERATURE}")

 Selected optimal temperature: 0.4


## Generate Results for Test CVs

In [23]:
# Create results directory
results_dir = Path('../results/gentle_roaster')
results_dir.mkdir(parents=True, exist_ok=True)

# Process each test CV
results = []

for idx in test_cv_indices:
    print(f"\n{'='*80}")
    print(f"Processing CV #{idx}")
    print(f"{'='*80}\n")
    
    # Format CV
    cv_text = format_cv_for_llm(df.iloc[idx])
    
    # Generate critique
    critique = roast_cv(cv_text, temperature=OPTIMAL_TEMPERATURE)
    
    # Save result
    result = {
        'cv_index': idx,
        'model': 'gentle_roaster',
        'temperature': OPTIMAL_TEMPERATURE,
        'timestamp': datetime.now().isoformat(),
        'cv_text': cv_text,
        'critique': critique
    }
    
    results.append(result)
    
    # Save to file
    output_file = results_dir / f"cv_{idx}_gentle.json"
    with open(output_file, 'w') as f:
        json.dump(result, f, indent=2)
    
    # Display result
    print(f" Original CV:")
    print("-"*80)
    print(cv_text)
    print(f"\n Gentle Critique:")
    print("-"*80)
    print(critique)
    print(f"\n Saved to: {output_file}")
    print(f"\n{'='*80}\n")

print(f"\n Generated {len(results)} gentle critiques")


Processing CV #0

 Original CV:
--------------------------------------------------------------------------------
CAREER OBJECTIVE:
Big data analytics working and database warehouse manager with robust experience in handling all kinds of data. I have also used multiple cloud infrastructure services and am well acquainted with them. Currently in search of role that offers more of development.

SKILLS:
['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapreduce', 'Spark', 'Java', 'Machine Learning', 'Cloud', 'Hdfs', 'YARN', 'Core Java', 'Data Science', 'C++', 'Data Structures', 'DBMS', 'RDBMS', 'Informatica', 'Talend', 'Amazon Redshift', 'Microsoft Azure']

EDUCATION:
Institution: ['The Amity School of Engineering & Technology (ASET), Noida']
Degree: ['B.Tech']
Major: ['Electronics']
Year: ['2019']

WORK EXPERIENCE:
Company: ['Coca-COla']
Position: ['Big Data Analyst']
Period: ['Nov 2019']
 to ['Till Date']
Responsibilities:
Technical Support
Troubleshooting
Collaboration
Documentation
System Mo

## Summary

This notebook demonstrated:
1.  Gentle CV critique prompt design
2.  Temperature tuning experiments & selection 
3.  Generation of critiques for test CVs
4.  Saving results for comparison

---

## Next: 03_medium_roaster.ipynb
More direct and honest feedback with balanced criticism.